# Simulation of Local Protocols

This notebook is meant to be used to simulate a local protocol supplemented by one bit of communication. The protocol is simulated using Monte-Carlo simulation.

In [1]:
# Import the required packages
import numpy as np
import math

First, let us define some functions

In [27]:
def random_vector(n):
    """Uniformly distributed random vector"""
    components = [np.random.normal() for i in range(n)]
    r = math.sqrt(sum(x*x for x in components))
    v = np.array([x/r for x in components])
    return v


def dot(vec_1, vec_2):
    """Dot product for two vectors"""
    return vec_1[0] * vec_2[0] + vec_1[1] * vec_2[1] + vec_1[2] * vec_2[2]


def outer_product(vec_1, vec_2):
    return np.outer(vec_1, vec_2).flatten()


def alice_protocol_1(vec_alice, lhv):
    """First protocol of Alice"""
    prob = 1/2 - 1/2 * np.sign(dot(vec_alice, lhv[0]))
    return np.array([prob, 1-prob])


def alice_protocol_2(vec_alice, lhv):
    """Second protocol of Alice"""
    return alice_protocol_1(vec_alice, lhv)


def bob_protocol_1(vec_bob, lhv):
    """First protocol of Bob"""
    prob = 1/2 + 1/2 * np.sign(dot(vec_bob, lhv[0] - lhv[1]))
    return np.array([prob, 1-prob])


def bob_protocol_2(vec_bob, lhv):
    """Second protocol of Bob"""
    prob = 1/2 + 1/2 * np.sign(dot(vec_bob, lhv[0] + lhv[1]))
    return np.array([prob, 1-prob])


def comm_protocol(vec_alice, lhv):
    """Protocol for the communication bit"""
    return 1/2 - 1/2 * np.sign(dot(vec_alice, lhv[0])) * np.sign(dot(vec_alice, lhv[1]))

Let us try to test the protocol. We print the joint expectation of the simulated protocol and the predicted theoretical value for a singlet.

In [21]:
theta = np.pi/4
n = 100000
alice = np.array([0.1, 0.9])
bob = np.array([0.3, 0.7])

print(outer_product(alice, bob))
# output is in the form of P(AB) -> (P(+1+1), P(+1-1), P(-1+1), P(-1-1))

[0.03 0.07 0.27 0.63]


Here let us now define new protocols. These protocols have the same marginals as the non-maximally entangled state but different joint expectation.

In [31]:
def probability_single_round(vec_alice, vec_bob, lhv):
    bit = comm_protocol(vec_alice, lhv)
    return bit * outer_product(alice_protocol_1(vec_alice, lhv), bob_protocol_1(vec_bob, lhv)) + \
        (1 - bit) * outer_product(alice_protocol_2(vec_alice, lhv), bob_protocol_2(vec_bob, lhv))
    
    
def probability(vec_alice, vec_bob, lhv_type="double-vector", n=10000):
    prob = np.array([0.0, 0.0, 0.0, 0.0])
    for i in range(n):
        if lhv_type == "single-vector":
            lhv = random_vector(3)
        elif lhv_type == "double-vector":
            lhv = (random_vector(3), random_vector(3))
        prob += 1/n * probability_single_round(vec_alice, vec_bob, lhv)
    return prob

In [33]:
probability(np.array([0, 1, 0]), np.array([0, 1, 0]))

array([0.    , 0.5043, 0.4957, 0.    ])